In [ ]:
%env OPENAI_API_KEY="keyhere"

In [ ]:
#!/bin/bash

helm upgrade --install argo-rollouts argo-rollouts \
  --repo https://argoproj.github.io/argo-helm \
  --version 2.37.6 \
  --namespace argo-rollouts \
  --create-namespace \
  --wait

kubectl apply -f- <<EOF
apiVersion: apps/v1
kind: Deployment
metadata:
  name: rollouts-demo
  labels:
    app: rollouts-demo
spec:
  replicas: 4
  selector:
    matchLabels:
      app: rollouts-demo
  template:
    metadata:
      labels:
        app: rollouts-demo
    spec:
      containers:
      - name: rollouts-demo
        image: argoproj/rollouts-demo:blue
        ports:
        - name: http
          containerPort: 8080
          protocol: TCP
  strategy:
    type: RollingUpdate
    rollingUpdate:
      maxSurge: 1
      maxUnavailable: 1
---
apiVersion: v1
kind: Service
metadata:
  name: rollouts-demo
  labels:
    app: rollouts-demo
spec:
  ports:
  - port: 80
    targetPort: http
    protocol: TCP
    name: http
  selector:
    app: rollouts-demo
apiVersion: gateway.networking.k8s.io/v1
kind: Gateway
metadata:
  name: gateway
spec:
  gatewayClassName: istio
  listeners:
  - name: default
    port: 80
    protocol: HTTP
    allowedRoutes:
      namespaces:
        from: All
---
kind: HTTPRoute
apiVersion: gateway.networking.k8s.io/v1beta1
metadata:
  name: rollouts-demo
spec:
  parentRefs:
    - name: gateway
  rules:
  - matches:
    - path:
        type: PathPrefix
        value: /  
    backendRefs:
    - name: rollouts-demo
      kind: Service
      port: 80
EOF

In [1]:
from autogen_agentchat.agents import AssistantAgent, UserProxyAgent
from autogen_agentchat.conditions import MaxMessageTermination, TextMentionTermination
from autogen_agentchat.teams import RoundRobinGroupChat
from autogen_agentchat.ui import Console
from autogen_ext.models.openai import OpenAIChatCompletionClient

from kagent.tools.argo import (
    CreateRolloutResource,
    GetRollout,
    PauseRollout,
    PromoteRollout,
    SetRolloutImage,
    StatusRollout,
    VerifyInstall,
)
from kagent.tools.k8s import GetPods, GetServices

model_client = OpenAIChatCompletionClient(
    model="gpt-4o",
)

argo_agent = AssistantAgent(
    "argo_agent",
    description="An agent for Argo Rollouts operations",
    tools=[CreateRolloutResource(), GetRollout(), PauseRollout(), PromoteRollout(), SetRolloutImage(), StatusRollout(), VerifyInstall()],
    model_client=model_client,
    system_message="""
    You are an Argo Rollouts agent. You know how to interact with Argo Rollouts in a Kubernetes cluster.
    Your job is to assist with Argo Rollouts operations. DO NOT MAKE UP ADDITIONAL AND UNNECESSARY SUBTASKS.
    Your tools are kubectl argo rollouts:
    - kubectl argo rollouts create
    - kubectl argo rollouts get
    - kubectl argo rollouts pause
    - kubectl argo rollouts resume
    - kubectl argo rollouts promote
    - kubectl argo rollouts status
    - kubectl argo rollouts abort
    - verify installation
    """,
)

k8s_agent = AssistantAgent(
    "k8s_agent",
    description="An agent for k8s operations",
    tools=[GetPods(), GetServices()],
    model_client=model_client,
    system_message="""
    You are a k8s agent. You know how to interact with a Kubernetes cluster.

    If you don't have any explicit tasks left to complete, return TERMINATE.
    """,
)

team = RoundRobinGroupChat(
    max_turns=3,
    participants=[k8s_agent, argo_agent],
    termination_condition=TextMentionTermination(text="TERMINATE"),
)

task = "Create an Argo Rollout to deploy a new version of the demo application with the color purple using the Kubernetes Gateway API"

# Use asyncio.run(...) if you are running this in a script.
await Console(team.run_stream(task=task))

---------- user ----------
Create an Argo Rollout to deploy a new version of the demo application with the color purple using the Kubernetes Gateway API
---------- k8s_agent ----------
Creating an Argo Rollout to deploy a new version of a demo application with a specific color involves defining the Argo Rollout resource in a YAML file. Below is an example of how you might define this rollout, along with annotations for the Kubernetes Gateway API for traffic management.

```yaml
apiVersion: argoproj.io/v1alpha1
kind: Rollout
metadata:
  name: demo-app
  namespace: default
spec:
  replicas: 3
  strategy:
    canary:
      steps:
        - setWeight: 25
        - pause: {}
        - setWeight: 50
        - pause: { duration: 600 }
        - setWeight: 100
  selector:
    matchLabels:
      app: demo
  template:
    metadata:
      labels:
        app: demo
    spec:
      containers:
      - name: demo-app
        image: demo-app:purple
        ports:
        - containerPort: 80
---
apiVe

TaskResult(messages=[TextMessage(source='user', models_usage=None, content='Create an Argo Rollout to deploy a new version of the demo application with the color purple using the Kubernetes Gateway API', type='TextMessage'), TextMessage(source='k8s_agent', models_usage=RequestUsage(prompt_tokens=284, completion_tokens=460), content="Creating an Argo Rollout to deploy a new version of a demo application with a specific color involves defining the Argo Rollout resource in a YAML file. Below is an example of how you might define this rollout, along with annotations for the Kubernetes Gateway API for traffic management.\n\n```yaml\napiVersion: argoproj.io/v1alpha1\nkind: Rollout\nmetadata:\n  name: demo-app\n  namespace: default\nspec:\n  replicas: 3\n  strategy:\n    canary:\n      steps:\n        - setWeight: 25\n        - pause: {}\n        - setWeight: 50\n        - pause: { duration: 600 }\n        - setWeight: 100\n  selector:\n    matchLabels:\n      app: demo\n  template:\n    meta

In [ ]:
print(team.dump_component().model_dump_json(indent=2))